In [1]:
# 📦 Step 1: Install Required Libraries (only once)
!pip install tensorflow pillow numpy matplotlib

In [ ]:
# 📁 Step 2: Extract Dataset
import zipfile
# Replace 'rice.zip' with your uploaded file name
with zipfile.ZipFile("rice.zip", 'r') as zip_ref:
    zip_ref.extractall("rice_dataset")

In [ ]:
# 👁️‍🗨️ Step 3: Setup and Import Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import os

In [ ]:
# 🔧 Setup
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 2
DATASET_DIR = "rice_dataset"  # Folder after extracting Kaggle zip

In [ ]:
# 📊 Step 4: Preprocess Dataset
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical'
)

val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',
    class_mode='categorical'
)

In [ ]:
# ✅ Display Classes
class_names = list(train_gen.class_indices.keys())
print("Classes:", class_names)

In [ ]:
# 🧠 Step 5: Build Model using MobileNetV2
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')  # Output layer
])

# ⚙️ Step 6: Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 📈 Step 7: Train the Model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)]
)

In [ ]:
# 💾 Step 8: Save the Model
model.save("rice_model.h5")
print("✅ Model saved as rice_model.h5")